In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
import os

from PIL import Image
import numpy as np 
import pandas as pd
from tqdm.auto import tqdm
import cv2
import sys
sys.path.append("/kaggle/usr/lib/siim_infer_helper_func/")
from siim_infer_helper_func import *

In [ ]:
df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
if df.shape[0] == 2477:
    tp1 = "/kaggle/input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm"
    tp2 = "/kaggle/input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm"
    fast_sub = True
    fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
                         ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
                         ['65761e66de9f_image', 'none 1 0 0 1 1'], 
                         ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
                       columns=['id', 'PredictionString'])
else:
    fast_sub = False
#fast_sub = False

In [ ]:
#import numpy as np
#import pydicom
#from pydicom.pixel_data_handlers.util import apply_voi_lut

study_size = (768,768)
image_size = (640,640)

split = 'test'
save_dir = f'/kaggle/tmp/{split}/'

os.makedirs(save_dir, exist_ok=True)

save_dir = f'/kaggle/tmp/{split}/study/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray(tp1)
    im = resize(xray, size1=study_size[0],size2=study_size[1])  
    study = '00086460a852' + '_study.png'
    im.save(os.path.join(save_dir, study))
    assert(tp1.split("/")[-3] == '00086460a852')
    xray = read_xray(tp2)
    im = resize(xray, size1=study_size[0],size2=study_size[1])  
    study = '000c9c05fd14' + '_study.png'
    im.save(os.path.join(save_dir, study))
    assert(tp2.split("/")[-3] == '000c9c05fd14')
else:   
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size1=study_size[0], size2=study_size[1])  
            study = dirname.split('/')[-2] + '_study.png'
            im.save(os.path.join(save_dir, study))

In [ ]:
image_id = []
dim0 = []
dim1 = []
splits = []
save_dir = f'/kaggle/tmp/{split}/image/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray(tp1)
    im = cv2.resize(xray,image_size,interpolation = cv2.INTER_AREA)
    cv2.imwrite(os.path.join(save_dir,'65761e66de9f_image.png'),im)
    image_id.append('65761e66de9f.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
    xray = read_xray(tp2)
    im = cv2.resize(xray,image_size,interpolation = cv2.INTER_AREA)
    cv2.imwrite(os.path.join(save_dir, '51759b5579bc_image.png'),im)
    image_id.append('51759b5579bc.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
else:
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = cv2.resize(xray,image_size,interpolation = cv2.INTER_AREA)
            cv2.imwrite(os.path.join(save_dir, file.replace('.dcm', '_image.png')),im)
            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            splits.append(split)
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})

# study level and 2-class

## make submit dataframe

In [ ]:
if fast_sub:
    df = fast_df.copy()
else:
    df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
id_laststr_list  = []
for i in range(df.shape[0]):
    id_laststr_list.append(df.loc[i,'id'][-1])
df['id_last_str'] = id_laststr_list

study_len = df[df['id_last_str'] == 'y'].shape[0]

## study level

In [ ]:
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
sub_df = sub_df[:study_len]
study_paths = f'/kaggle/tmp/{split}/study/' + sub_df['id'] +'.png'

sub_df['negative'] = 0
sub_df['typical'] = 0
sub_df['indeterminate'] = 0
sub_df['atypical'] = 0

dfstudy_tf = sub_df.copy()
dfstudy_torch = sub_df.copy()
dfstudy = sub_df.copy()
study_label_cols = sub_df.columns[2:]

if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
sub_df = sub_df[study_len:].reset_index(drop=True)
image_paths = f'/kaggle/tmp/{split}/image/' + sub_df['id'] +'.png'
sub_df['none'] = 0

df_2class_tf = sub_df.copy()
df_2class_touch = sub_df.copy()
df_2class = sub_df.copy()


class2_label_cols = sub_df.columns[2]

## model and weight paths

In [ ]:
study_tf_mpaths = [
    "/kaggle/input/bestmodeldataset/fold0_v2l_0.828.h5",
    "/kaggle/input/bestmodeldataset/fold1_v2l_0.828.h5",
]

binaryclass_tf_mpaths = [
    "/kaggle/input/2classtfv2m/model0.h5",
    "/kaggle/input/2classtfv2m/model1.h5",
    "/kaggle/input/2classtfv2m/model3.h5",
    "/kaggle/input/2classtfv2m/model4.h5",
]


binaryclass_tf_mpaths_b5 =[
    "/kaggle/input/2class-tf/model0.h5",
    "/kaggle/input/2class-tf/model1.h5",
    "/kaggle/input/2class-tf/model3.h5",
]

maskwpath = ["../input/covid19-effneteffdet/auxlossv2m_weight/pretrained_model_0_0.834.bin",
             "../input/covid19-effneteffdet/auxlossv2m_weight/pretrained_model_2_0.854.bin",
             "../input/covid19-effneteffdet/auxlossv2m_weight/pretrained_model_3_0.832.bin",
             "../input/covid19-effneteffdet/auxlossv2m_weight/pretrained_model_4_0.849.bin",
            ]

bcemaskpath = [
      "../input/covid19-effneteffdet/auxlossv2m_bcemodel/fold0_0.837.bin",
      "../input/covid19-effneteffdet/auxlossv2m_bcemodel/fold2_0.857.bin",
      "../input/covid19-effneteffdet/auxlossv2m_bcemodel/fold3_0.851.bin",
      "../input/covid19-effneteffdet/auxlossv2m_bcemodel/fold4_0.855.bin",
             ]


nonewpath = ["../input/covid19-effneteffdet/2class_v2m/fold0_0.924.bin",
             "../input/covid19-effneteffdet/2class_v2m/fold3_0.921.bin",
             "../input/covid19-effneteffdet/2class_v2m/fold4_0.923.bin",
             "../input/covid19-effneteffdet/cutmix_fold0_0.934.bin",
             "../input/covid19-effneteffdet/cutmix_fold1_0.931.bin",
             "../input/covid19-effneteffdet/cutmix_fold4_0.932.bin",
            ]


# tensorflow

In [ ]:
#prepare tensorflow module

!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

import os

import efficientnet.tfkeras as efn
import tensorflow as tf

In [ ]:
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16
test_decoder0 = build_decoder(with_labels=False, target_size=(690, 690), ext='png')
test_decoder1 = build_decoder(with_labels=False, target_size=(710, 710), ext='png')
dtest1 = build_dataset(
    study_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder0
)

dtest2 = build_dataset(
    study_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder1
)



with strategy.scope():
    
    models = []
    for mpath in study_tf_mpaths:
        model = tf.keras.models.load_model(mpath,custom_objects={'KerasLayer': KerasLayerWrapperv2l})
        models.append(model)
        del model

tmp1 = sum([model.predict(dtest1, verbose=1) for model in models]) / len(models)
tmp2 = sum([model.predict(dtest2, verbose=1) for model in models]) / len(models)

dfstudy_tf[study_label_cols] = (tmp1 + tmp2)/2

In [ ]:
test_decoder = build_decoder(with_labels=False, target_size=(660,660), ext='png')
dtest = build_dataset(
    image_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

with strategy.scope():
    
    models = []
    models2 = []
    for mpath in binaryclass_tf_mpaths:
        
        model = tf.keras.models.load_model(mpath,custom_objects={'KerasLayer': KerasLayerWrapperv2m})
        models.append(model)
        del model
    
    for mpath in binaryclass_tf_mpaths_b5:
        model = tf.keras.models.load_model(mpath)
        models2.append(model)
        del model
        
                
tmp1 = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)

del models

test_decoder = build_decoder(with_labels=False, target_size=(512,512), ext='png')
dtest = build_dataset(
    image_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

tmp2 = sum([model.predict(dtest, verbose=1) for model in models2]) / len(models2)
del models2

df_2class_tf[class2_label_cols] = (tmp1+tmp2)/2
df_2class_tf = df_2class_tf.reset_index(drop=True)

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

# Pytorch

In [ ]:
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import torch
import torch.nn as nn

import sys
sys.path.insert(0, "../input/timm-pytorch-image-models/pytorch-image-models-master/")
import timm

device = 'cuda'

# Pytorch studylevel

In [ ]:
def get_valid_transforms(size):
    return A.Compose(
        [
            A.Resize(height=size, width=size, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        
    )

def augtransform(size):
    return A.Compose(
        [
            A.RandomSizedCrop(min_max_height=[740,768],
                         height=740,
                         width=740,
                         p=0.5),
            A.CLAHE(clip_limit=(1,1.5),p=0.5),
            A.RandomBrightnessContrast(
                    brightness_limit=0.05, 
                    contrast_limit=0.05,
                    p=0.5),
            A.HorizontalFlip(p=0.25),
            A.VerticalFlip(p=0.25),
            A.Rotate(limit=5,p=0.25),
            A.Resize(height=size, width=size, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        
    )

DATA_ROOT_PATH = '/kaggle/tmp/test/study'

class DatasetRetriever(Dataset):

    def __init__(self, df, path):
        super().__init__()
        self.df = df
        self.path = path

    def __getitem__(self, index: int):
        path = self.path[index]
        image_id = self.df.id[index]
        image = cv2.imread(path,cv2.IMREAD_COLOR)        
        
        image0 = get_valid_transforms(512)(image=image)["image"]
        image1 = augtransform(506)(image=image)["image"]
        image2 = augtransform(520)(image=image)["image"]
        image3 = augtransform(510)(image=image)["image"]

        image0 = image0/255.0
        image1 = image1/255.0
        image2 = image2/255.0
        image3 = image3/255.0
        
        return image0,image1,image2,image3,image_id

    def __len__(self) -> int:
        return self.df.shape[0]

In [ ]:
dataset = DatasetRetriever(
    df= dfstudy_torch,
    path = study_paths,
)

data_loader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    drop_last=False,
)

In [ ]:
mmodels = []
bcemodels = []

for wpath in maskwpath:
    print(wpath)
    model = Net().to(device)
    model.load_state_dict(torch.load(wpath))
    model.eval()
    mmodels.append(model)
    del model

for bcepath in bcemaskpath:
    bcemodel = Net().to(device)
    bcemodel.load_state_dict(torch.load(bcepath))
    bcemodel.eval()
    bcemodels.append(bcemodel)
    del bcemodel

In [ ]:
image_ids = []
outputs = np.empty((0,4))

with torch.no_grad():
    for image0,image1,image2,image3, image_id, in data_loader:
        image0 = image0.cuda().float()
        image1 = image1.cuda().float()
        image2 = image2.cuda().float()
        image3 = image3.cuda().float()
        for i,mmodel in enumerate(mmodels):
            if i==0: 
                mout = (
                        bcemodels[0](image0)[0].sigmoid() +
                        bcemodels[1](image1)[0].sigmoid() +
                        bcemodels[2](image2)[0].sigmoid() +
                        bcemodels[3](image3)[0].sigmoid())/4
            else:
                mout +=(mmodel(image0)[0].softmax(dim=1) +
                        mmodel(image1)[0].softmax(dim=1) +
                        mmodel(image2)[0].softmax(dim=1) +
                        mmodel(image3)[0].softmax(dim=1))/4
    
        mout = mout/len(mmodels)
        
        outputs = np.append(outputs,mout.cpu().detach().numpy(),axis=0)
        image_ids = np.append(image_ids,image_id)
print(outputs.shape)
print(image_ids.shape)
dfstudy_torch.id = image_ids
dfstudy_torch[study_label_cols] = outputs

del bcemodels
del mmodels

In [ ]:
dfstudy[study_label_cols] = (dfstudy_tf[study_label_cols] + dfstudy_torch[study_label_cols])/2
display(dfstudy_torch,dfstudy_tf)
display(dfstudy)

In [ ]:
for i in range(study_len):
    negative = dfstudy.loc[i,'negative']
    typical = dfstudy.loc[i,'typical']
    indeterminate = dfstudy.loc[i,'indeterminate']
    atypical = dfstudy.loc[i,'atypical']
    dfstudy.loc[i, 'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'
df_study = dfstudy[["id","PredictionString"]]
display(df_study)

# Pytorch 2-class

In [ ]:
def get_valid_transforms(size):
    return A.Compose(
        [
            A.Resize(height=size, width=size, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        
    )

def augtransform(size):
    return A.Compose(
        [
            A.RandomSizedCrop(min_max_height=[610,640],
                         height=600,
                         width=600,
                         p=0.5),
            A.CLAHE(clip_limit=(1,1.5),p=0.5),
            A.RandomBrightnessContrast(
                    brightness_limit=0.05, 
                    contrast_limit=0.05,
                    p=0.5),
            A.HorizontalFlip(p=0.25),
            A.VerticalFlip(p=0.25),
            A.Rotate(limit=5,p=0.25),
            A.Resize(height=size, width=size, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        
    )

class DatasetRetriever(Dataset):

    def __init__(self, df, path):
        super().__init__()
        self.df = df
        self.path = path

    def __getitem__(self, index: int):
        path = self.path[index]
        image_id = self.df.id[index]
        image = cv2.imread(path,cv2.IMREAD_COLOR)        
        
        image0 = get_valid_transforms(512)(image=image)["image"]
        image1 = augtransform(506)(image=image)["image"]
        image2 = augtransform(520)(image=image)["image"]
        image3 = augtransform(510)(image=image)["image"]

        image0 = image0/255.0
        image1 = image1/255.0
        image2 = image2/255.0
        image3 = image3/255.0
        
        return image0,image1,image2,image3,image_id

    def __len__(self) -> int:
        return self.df.shape[0]

In [ ]:
from timm.models.efficientnet import *
class Net2class(nn.Module):
    def __init__(self):
        super(Net2class, self).__init__()

        e = tf_efficientnetv2_m_in21k(pretrained=False, drop_rate=0.3, drop_path_rate=0.2,in_chans=3)
        self.b0 = nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1,
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.gp = e.global_pool
        self.b8 = nn.Sequential(
            e.conv_head, #384, 1536
            e.bn2,
            e.act2,
        )

        self.logit = nn.Linear(1280,1)
        self.mask = nn.Sequential(
            nn.Conv2d(176, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    # @torch.cuda.amp.autocast()
    def forward(self, image):
        batch_size = len(image)
        x = image

        x = self.b0(x) 
        x = self.b1(x) 
        x = self.b2(x) 
        x = self.b3(x) 
        x = self.b4(x) 
        x = self.b5(x) 
        #------------
        mask = self.mask(x)
        #-------------
        x = self.b6(x) 
        x = self.b7(x) 
        x = self.b8(x) 
        x = self.gp(x)
        x = F.dropout(x, 0.1, training=True)
        logit = self.logit(x)
        return logit, mask

In [ ]:
nonemodels = []
for wpath in nonewpath:
    print(wpath)
    model = Net2class().to(device)
    model.load_state_dict(torch.load(wpath))
    model.eval()
    nonemodels.append(model)

In [ ]:
dataset = DatasetRetriever(
    df=df_2class_touch,
    path = image_paths,
)

data_loader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=False,
    num_workers=2,
    drop_last=False,
)

In [ ]:
image_ids = []
noneout = []

with torch.no_grad():
    for image0,image1,image2,image3, image_id, in data_loader:
        image0 = image0.cuda().float()
        image1 = image1.cuda().float()
        image2 = image2.cuda().float()
        image3 = image3.cuda().float()
        for i,nmodel in enumerate(nonemodels):
            if i==0: 
                nout = (nmodel(image0)[0].sigmoid() +
                        nmodel(image1)[0].sigmoid() +
                        nmodel(image2)[0].sigmoid() +
                        nmodel(image3)[0].sigmoid())/4
            else:
                nout += (nmodel(image0)[0].sigmoid() +
                        nmodel(image1)[0].sigmoid() +
                        nmodel(image2)[0].sigmoid() +
                        nmodel(image3)[0].sigmoid())/4


        nout = nout/len(nonemodels)
        noneout = np.append(noneout,nout.cpu().detach().numpy())
        image_ids = np.append(image_ids,image_id)
print(noneout.shape)
print(image_ids.shape)
df_2class_touch[class2_label_cols] = noneout
df_2class_touch = df_2class_touch.reset_index(drop=True)

In [ ]:
df_2class[class2_label_cols] = (df_2class_tf[class2_label_cols] + df_2class_touch[class2_label_cols])/2
display(df_2class_tf,df_2class_touch)
df_2class

In [ ]:
del model
del nonemodels

# image classification end 

# image level start

## image level path list

In [ ]:
weights_dir = ['/kaggle/input/yolo-bestresult/best_fold0.pt',
               '/kaggle/input/yolo-bestresult/best_fold1.pt',
               '/kaggle/input/yolo-bestresult/best_fold2.pt',
               '/kaggle/input/yolo-bestresult/best_fold3.pt',
               '/kaggle/input/yolo-bestresult/best_fold4.pt',]

paths = ["/kaggle/input/covid19-effneteffdet/d3_image640/fold1-d3_0.630.bin",
         "/kaggle/input/covid19-effneteffdet/d3_image640/fold4-d3_0.660.bin",
         "/kaggle/input/covid19-effneteffdet/fold0-cspresdext50pan_0.650.bin",
         "/kaggle/input/covid19-effneteffdet/fold3-cspresdext50pan_0.634.bin",
         "/kaggle/input/covid19-effneteffdet/fold4_cspresdext50pan-0.672.bin",
        ]

model_names = [
    "tf_efficientdet_d3",
    "tf_efficientdet_d3",
    "cspresdext50pan",
    "cspresdext50pan",
    "cspresdext50pan",
]

det_weights = [1,1,1,1,1,1]
weight_detyolo = [1,2]

meta = meta[meta['split'] == 'test']
if fast_sub:
    test_df = fast_df.copy()
else:
    test_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
test_df = df[study_len:].reset_index(drop=True) 
meta['image_id'] = meta['image_id'] + '_image'
meta.columns = ['id', 'dim0', 'dim1', 'split']
test_df = pd.merge(test_df, meta, on = 'id', how = 'left')

test_df

# yolo

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch


dim = 640 #1024, 256, 'original'
test_dir = f'/kaggle/tmp/{split}/image'


shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5') # install dependencies

!python detect.py \
--weights {weights_dir[0]} {weights_dir[1]} {weights_dir[2]} {weights_dir[3]} {weights_dir[4]} \
--img 640\
--conf 0.001\
--iou 0.5\
--source $test_dir\
--name infer_fold \
--save-txt --save-conf --exist-ok --augment

labelpaths = glob("runs/detect/infer_fold/labels/*")

predbox_ens = []
image_ids = []
score = []
box = []
for file_path in tqdm(labelpaths):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    data = np.concatenate([data[:, :2], yolo2a(data[:, 2:])],axis=1)
    
    if fast_sub:
        predbox_ens.append(norm2hw(h, w, data[:,2:]))
    
    
    image_ids.append(image_id)
    score.append(data[:,1])
    box.append(data[:,2:])

pred_df_yolo = pd.DataFrame({'id':image_ids,'score':score,'label':1,'box':box})

if fast_sub:
    display(pred_df_yolo)
    box_plot(tp1,predbox_ens[0])
    box_plot(tp2,predbox_ens[1])

# effdet

In [ ]:
!pip install --no-deps '/kaggle/input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl'
!pip install "/kaggle/input/effdet-latestvinbigdata-wbf-fused/omegaconf-2.0.6-py3-none-any.whl"
import sys
sys.path.insert(0, "/kaggle/input/effdet-latestvinbigdata-wbf-fused/efficientdet-pytorch/")
sys.path.insert(0, "/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master/")
sys.path.insert(0, "/kaggle/input/weightedboxesfusion")

In [ ]:
import torch
import os

import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from effdet import get_efficientdet_config, EfficientDet, DetBenchPredict
from effdet.efficientdet import HeadNet
from glob import glob
import gc
from ensemble_boxes import *

def get_valid_transforms():
    return A.Compose(
        [
            #A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        
    )

DATA_ROOT_PATH = '/kaggle/tmp/test/image'

class DatasetRetriever(Dataset):

    def __init__(self, df, transforms=None):
        super().__init__()
        self.df = df
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.df.id.values[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.png',cv2.IMREAD_COLOR)
        dim0 = self.df.dim0.values[index]
        dim1 = self.df.dim1.values[index]
        image = image.astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        
        return sample['image'], image_id, dim0, dim1

    def __len__(self) -> int:
        return self.df.shape[0]
    
dataset = DatasetRetriever(
    df=test_df,
    transforms=get_valid_transforms()
)

data_loader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    drop_last=False,
)

def load_net(checkpoint_path,model_name):
    config = get_efficientdet_config(model_name)
    config.image_size = [640,640]
    config.norm_kwargs=dict(eps=.001, momentum=.01)
    net = EfficientDet(config, pretrained_backbone=False)
    net.reset_head(num_classes=1)
    net.class_net = HeadNet(config, num_outputs=config.num_classes)

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])
    del checkpoint
    gc.collect()
    evalnet = DetBenchPredict(net)
    evalnet.eval();
    return evalnet.cuda()

In [ ]:
nets = []
for path, mname in zip(paths,model_names):
    print(path)
    model = load_net(path,mname)
    nets.append(model)

In [ ]:
def make_predictions(images, nets, score_threshold=0.001):
    images = images.cuda().float()
    predictions = []
    dets = []
    with torch.no_grad():
        for model in nets:
            dets.append(model(images))
        for i in range(images.shape[0]):
            boxes = [det[i].detach().cpu().numpy()[:,[1,0,3,2]]/640.0 for det in dets]
            scores = [det[i].detach().cpu().numpy()[:,4] for det in dets]
            label = [det[i].detach().cpu().numpy()[:,5] for det in dets]
            
            boxes, scores, label  = weighted_boxes_fusion(boxes,
                                                          scores,
                                                          label,
                                                          weights=det_weights,
                                                          iou_thr=0.6,
                                                          conf_type='avg',
                                                          skip_box_thr=0.001)
            
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]
            scores = scores[indexes]
            label = label[indexes]
            predictions.append({
                'boxes': boxes,
                'scores': scores,
                'label': label
            })
    return predictions

In [ ]:
results = []
boxess = []
for j,(images, image_ids, dim0, dim1) in enumerate(data_loader):
    #print(images.shape)
    predictions = make_predictions(images, nets)
    for i,prediction in enumerate(predictions):
        boxes = prediction['boxes']
        scores = prediction['scores']
        label = prediction['label']
        image_id = image_ids[i]
        
        index = pred_df_yolo[pred_df_yolo.id==image_id].index
        yolorow = pred_df_yolo.iloc[index]
        boxes_y = yolorow.box.values[0]
        scores_y = yolorow.score.values[0]
        labels_y = label
        ens_box = [boxes,boxes_y]
        ens_score = [scores,scores_y]
        ens_label = [label,labels_y]
        
        
        
        boxes, scores, label  = weighted_boxes_fusion(ens_box,
                                                      ens_score,
                                                      ens_label,
                                                      weights=weight_detyolo,
                                                      iou_thr=0.6,
                                                      conf_type='avg',
                                                      skip_box_thr=0.001)
        
        boxes[:,0], boxes[:,2] = boxes[:,0]*dim1[i].item(), boxes[:,2]*dim1[i].item()
        boxes[:,1], boxes[:,3] = boxes[:,1]*dim0[i].item(), boxes[:,3]*dim0[i].item()  
        boxes = boxes.astype(np.int32)
        if fast_sub:
            boxess.append(boxes)
        result = {
            'id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }
        results.append(result)
pred_df = pd.DataFrame(results, columns=['id', 'PredictionString'])
pred_df

In [ ]:
pred_df['none'] = df_2class['none'] 
pred_df

In [ ]:
for i in range(sub_df.shape[0]):
    if pred_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
        pred_df.loc[i,'PredictionString'] = pred_df.loc[i,'PredictionString'] + ' none ' + str(pred_df.loc[i,'none']) + ' 0 0 1 1'
pred_df = pred_df[['id', 'PredictionString']]   
pred_df

In [ ]:
if fast_sub:
    box_plot(tp1,boxess[0])
    box_plot(tp2,boxess[1])

In [ ]:
pred_df

In [ ]:
#df_study = df[:study_len]
df_study = df_study.append(pred_df).reset_index(drop=True)
df_study = df_study[['id','PredictionString']]
df_study

In [ ]:
df_study.to_csv('/kaggle/working/submission.csv',index = False) 
df_study